<a href="https://colab.research.google.com/github/chinnusai25/cvit-Workshop2020/blob/master/Day9(June4)(AfterNoonSession_Basic_MLP_in_PyTorch).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install torch
!pip3 install torchvision

In [0]:
import torch
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch import optim

In [0]:
# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,)),
                              ])

In [4]:
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

Extracting /root/.pytorch/MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to /root/.pytorch/MNIST_data/MNIST/raw


Extracting /root/.pytorch/MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to /root/.pytorch/MNIST_data/MNIST/raw


Extracting /root/.pytorch/MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to /root/.pytorch/MNIST_data/MNIST/raw


Extracting /root/.pytorch/MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /root/.pytorch/MNIST_data/MNIST/raw
Processing...
Done!


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


In [0]:
testset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

In PyTorch we usually process multiple samples at the same time. Usually we stack up each training vector along rows to create $X$ matrix.
\begin{equation}
X = \begin{pmatrix}
- & - & x_1 & - & -\\
. & . & . & . & .\\
. & . & . & . & .\\
. & . & . & . & .\\
. & . & . & . & .\\
. & . & . & . & .\\
- & - & x_{batch-size} & - & -\\
\end{pmatrix}
\end{equation}

torch.nn.Linear class when instantiated creates an object with a randomly initialised $W$ matrix of dimensions(in_class features, out_class features) and another randomly initialised matrix $b$ of dimensions(out_class features, 1). These dimensions are passed as arguments to the constructor during creation of object. When the forward function of the created object is invoked with parameters $X$, it applies linear transformation $XW + b$ and returns the appropriate matrix

In [6]:
# Build a Neural Network
class MLP(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = torch.nn.Linear(784, 128)
        self.fc2 = torch.nn.Linear(128, 64)
        self.fc3 = torch.nn.Linear(64, 10)
        self.sigmoid = torch.nn.Sigmoid()
        self.softmax = torch.nn.Softmax(dim = 1)
        self.relu = torch.nn.ReLU()
    def forward(self, x):
        x = x.view(x.shape[0], -1)
        x = self.fc1.forward(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.softmax(x)

        return x

In [0]:
# Create an object of class MLP
model = MLP()

# Define the loss
criterion = torch.nn.CrossEntropyLoss()

# Optimizers require the parameters to optimize and a learning rate
optimizer = optim.SGD(model.parameters(), lr = 0.05)

In [8]:
epochs = 100
for e in range(epochs):
    print("Epoch ", (e + 1))
    running_loss = 0
    for images, labels in trainloader:
        optimizer.zero_grad()
        logits = model.forward(images)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    else:
        accuracy = 0
        with torch.no_grad():
            for images, labels in testloader:
                logit = model(images)
                ps = logit#torch.exp(logit)
                top_p, top_class = ps.topk(1, dim = 1)
                equals = top_class == labels.view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor))
        print("Loss on training set: ", running_loss)
        print("Accuracy on test set: ", end = " ")
        print((accuracy / len(testloader)).data.numpy())

Epoch  1
Loss on training set:  1884.5094249248505
Accuracy on test set:  0.73178744
Epoch  2
Loss on training set:  1621.2596442699432
Accuracy on test set:  0.75318474
Epoch  3
Loss on training set:  1604.2683271169662
Accuracy on test set:  0.7579618
Epoch  4
Loss on training set:  1598.4794297218323
Accuracy on test set:  0.7606489
Epoch  5
Loss on training set:  1594.994345664978
Accuracy on test set:  0.7627388
Epoch  6
Loss on training set:  1592.4725897312164
Accuracy on test set:  0.76363456
Epoch  7
Loss on training set:  1590.110810995102
Accuracy on test set:  0.76144505
Epoch  8
Loss on training set:  1588.249595284462
Accuracy on test set:  0.76592356
Epoch  9
Loss on training set:  1586.6769217252731
Accuracy on test set:  0.7681131
Epoch  10
Loss on training set:  1584.892177581787
Accuracy on test set:  0.7650279
Epoch  11
Loss on training set:  1583.45898771286
Accuracy on test set:  0.76671976
Epoch  12
Loss on training set:  1582.1680119037628
Accuracy on test set: 